# SurrealDB

>[SurrealDB](https://surrealdb.com/) is an end-to-end cloud-native database designed for modern applications, including web, mobile, serverless, Jamstack, backend, and traditional applications. With SurrealDB, you can simplify your database and API infrastructure, reduce development time, and build secure, performant apps quickly and cost-effectively.
>
>**Key features of SurrealDB include:**
>
>* **Reduces development time:** SurrealDB simplifies your database and API stack by removing the need for most server-side components, allowing you to build secure, performant apps faster and cheaper.
>* **Real-time collaborative API backend service:** SurrealDB functions as both a database and an API backend service, enabling real-time collaboration.
>* **Support for multiple querying languages:** SurrealDB supports SQL querying from client devices, GraphQL, ACID transactions, WebSocket connections, structured and unstructured data, graph querying, full-text indexing, and geospatial querying.
>* **Granular access control:** SurrealDB provides row-level permissions-based access control, giving you the ability to manage data access with precision.
>
>View the [features](https://surrealdb.com/features), the latest [releases](https://surrealdb.com/releases), and [documentation](https://surrealdb.com/docs).

This notebook shows how to use functionality related to the `SurrealDBStore`.

## Basic Example

Run SurrealDB locally with:

```shell
$ surreal start -u root -p root
```

Uncomment the right line below to install the dependencies for this example:

In [1]:
# -- Using pip
# %pip install --upgrade --quiet langchain langchain-community langchain_huggingface surrealdb
# -- Using uv
# !uv pip install --upgrade --quiet langchain langchain-community langchain_huggingface surrealdb
# -- Local packages
# !uv pip install --upgrade --quiet langchain /Users/martin/repos/langchain-community/libs/community langchain_huggingface /Users/martin/repos/surrealdb.py


### Prepare the data and the embeddings model

In [2]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import SurrealDBStore
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import CharacterTextSplitter

documents = TextLoader("../../how_to/state_of_the_union.txt").load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

model_name = "sentence-transformers/all-mpnet-base-v2"
embeddings = HuggingFaceEmbeddings(model_name=model_name)

### Connect with SurrealDB and add the documents

In [4]:
from surrealdb import Surreal

conn = Surreal("ws://localhost:8000/rpc")
conn.signin({"username": "root", "password": "root"})
conn.use("langchain", "test")
db = SurrealDBStore(embeddings, conn)

# delete all existing documents from the vectorstore collection
db.delete()

# add documents to the vectorstore
ids = db.add_documents(docs)

# document ids of the added documents
ids[:5]

['jyty8yknmcuyueil5jq6',
 'fy8waavudl7sr8ht5256',
 'i4aihlpmo67kwfiwoiyo',
 'yydq9gk3h5f80yb9ed2h',
 'c264vm68zedbjd0a6vjp']

### Similarity search

In [6]:
query = "What did the president say about Ketanji Brown Jackson"
docs = db.similarity_search(query)

In [7]:
print(docs[0].page_content)

Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. 

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.


### Similarity search with score

The returned distance score is cosine distance. Therefore, a lower score is better.

In [8]:
docs = db.similarity_search_with_score(query)

In [9]:
docs[0]

(Document(id='q668bujtsv5p8n6mtzu8', metadata={'source': '../../how_to/state_of_the_union.txt'}, page_content='Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. \n\nTonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. \n\nOne of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. \n\nAnd I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.'),
 0.39839559039493777)